## What is Docker?

Docker is a set of platform as a service (PaaS) products that use OS-level virtualization to deliver software in packages called containers. [wikipedia](https://en.wikipedia.org/wiki/Docker_(software))

### Containers
OS-level virtualization is an operating system (OS) paradigm in which the kernel allows the existence of multiple isolated user space instances, called containers (LXC, Solaris containers, Docker, Podman), zones (Solaris containers), virtual private servers (OpenVZ), partitions, virtual environments (VEs), virtual kernels (DragonFly BSD), or jails (FreeBSD jail or chroot jail).[1] Such instances may look like real computers from the point of view of programs running in them. A computer program running on an ordinary operating system can see all resources (connected devices, files and folders, network shares, CPU power, quantifiable hardware capabilities) of that computer. However, programs running inside of a container can only see the container's contents and devices assigned to the container. [wikipedia](https://en.wikipedia.org/wiki/OS-level_virtualization)

![](https://geekflare.com/wp-content/uploads/2019/09/traditional-vs-new-gen.png)

![](https://containerjournal.com/wp-content/uploads/2020/06/Orasicontainerpic1.png)

### Advanced understanding of containers
![](https://opensource.com/sites/default/files/1linuxtechs.png)

![](https://www.studytrails.com/wp-content/uploads/2019/04/namespaces-1024x576.jpg)

### Basic docker commands
1. `docker run` -> Running a simple container (docker create + docker start). Example: `docker run hello-world` 
2. `docker ps` -> Get the running containers. `docker ps -a` -> Get the running and stopping containers.
3. `docker pull` -> Pull a container image from a registry. Example: `docker pull ubuntu`
4. `docker push` -> Push a container image to a registry. Example: `docker push blopezp007/my-image:0.0.1`
5. `docker cp` -> Copy a file/directory to a running container. Example: `docker cp my-file.txt container_name`
6. `docker exec` -> Executes a command in a running container. Example: `docker exec container_name cat /etc/hosts`
7. `docker create` -> Only creates a new container. Example: `docker create ubuntu --name my_container`
8. `docker images` -> List container images downloaded. 
9. `docker commit` -> Save the state of a container into an image. Example: `docker commit my_container my_image:v1`
10. `docker build` -> Build a container from a Dockerfile and save it as image. Example: `docker build . -t my_image:v2 -f Dockerfile`. The dot is the context for the container.
11. `docker image` -> Manage container images. 
12. `docker export` -> Export a container into a tarball file. Example: `docker export my_container -o my-container.tar`
13. `docker save` -> Export one or more images into a tarball file. Example: `docker save busybox ubuntu -o images.tar`
14. `docker import` -> Import a tarball into an image. You need to provide a tag. Example: `docker import my-container.tar my-image:exported`
15. `docker load` -> Load one or more images from a tarball file. Example: `docker load -i images.tar`
16. `docker rm` -> Remove a container stopped, you can remove a container running using the -f flag. Example: `docker rm my_container_running -f`
17. `docker rmi/docker image rm` -> Remove a container image from the host.
18. `docker login/logout` -> Login/Logout from a registry.
19. `docker start/stop/restart` -> Start/Stop/restart one or more containers.
20. `docker search` -> Search for a container image.
21. `docker kill` -> Kill one or more containers.
22. `docker diff` -> Inspect changes to files or directories on a container's filesystem
23. `docker top` -> Show the running processes.
24. `docker stats` -> Show resource usage for the running containers.
25. `docker history` -> Show the history for a container image (layers).
26. `docker inspect` -> Inspect a running container.
27. `docker volume` -> Manage volumes.
28. `docker network` -> Manage networks.
29. `docker container` -> Manage containers.
30. `docker secret/config` -> Manage configuration and secrets.
31. `docker scan` -> Security scan for a container image.
32. `docker swarm/stack/node` -> Docker swarm commands.
33. `docker buildx` -> Build container images.
34. `docker attach` -> Attach to a running container.
35. `docker system` -> Manage docker and disk space.
36. `docker logs` -> View stream logs from a container.

## docker build vs docker buildx build
The `docker buildx build` command supports features available for `docker build`, including features such as outputs configuration, inline build caching, and specifying target platform. In addition, Buildx also supports new features that are not yet available for regular `docker build` like building manifest lists, distributed caching, and exporting build results to OCI image tarballs. [Reference](https://github.com/docker/buildx)

- [docker volumes](https://docs.docker.com/storage/volumes/)
   > - Volume Mount (`docker volume create`)
   > - Bind Mount (Mount a directory or file from the docker host into the container)
- [docker network](https://docs.docker.com/network/)
   > - Bridge (use its own network `docker0`)
   > - Host (Use the docker host network, you don't need to use port forwarding)
   > - None (disable networking)


![](https://miro.medium.com/max/720/1*WKiEgPXO8XXppoqgr7ZVQA.png)

![](https://miro.medium.com/max/720/1*pxgEmyU1LZIfHENlBmFDHQ.png)

![](https://miro.medium.com/max/720/1*MiGJxyhghBXBZcim9LSZQQ.png)

## BUILD CONTAINERS
[Dockerfile Reference](https://docs.docker.com/engine/reference/builder/)

### .dockerignore
The .gitignore for the docker build command. Exclude files or directories included in the root context.

#### Dockerfile example:

```dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9-slim AS builder

WORKDIR /app

COPY requirements.txt ./
RUN --mount=type=cache,target=/root/.cache/pip 
    pip install -r requirements.txt

COPY ./app ./app

FROM builder as dev-envs

RUN <<EOF
apt-get update
apt-get install -y --no-install-recommends git
EOF

RUN <<EOF
useradd -s /bin/bash -m vscode
groupadd docker
usermod -aG docker vscode
EOF
# install Docker tools (cli, buildx, compose)
COPY --from=gloursdocker/docker / /
```

#### [MULTISTAGE BUILD:](https://docs.docker.com/build/building/multi-stage/)
```dockerfile
FROM golang:1.16 [AS builder]
WORKDIR /go/src/github.com/alexellis/href-counter/
RUN go get -d -v golang.org/x/net/html  
COPY app.go ./
RUN CGO_ENABLED=0 go build -a -installsuffix cgo -o app .

FROM alpine:latest  
RUN apk --no-cache add ca-certificates
WORKDIR /root/
COPY --from=0 /go/src/github.com/alexellis/href-counter/app ./
# COPY --from=builder /go/src/github.com/alexellis/href-counter/app ./
CMD ["./app"]
```

#### ADD vs COPY:
- ADD -> Add files, directories from a URL, host path and uncompress tarball files in the build. (Impredecible)
- COPY -> Add files, directories from host path only.

Preferred COPY and if you need to download from a URL or uncompress a tarball use RUN and verify the integrity of that file.

#### CMD vs ENTRYPOINT:
- CMD -> Command line or arguments that you can modify when run a container.
- ENTRYPOINT -> Command using when running the container, you cannot modify it, if you need to do you need to pass the `--entrypoint` flag to the docker run command.

![](https://miro.medium.com/max/720/1*ZyDop_nTtu26CVpz5uFQUg.png)

[Caching](https://docs.docker.com/build/building/cache/)

## docker-compose

We are using docker-compose if we need to run an application with a multiple services, for example a wordpress web site, that needs a mysql/postgresql database to save data. [Reference](https://docs.docker.com/compose/compose-file/compose-file-v3/)

```yaml
services:
  db:
    image: mariadb:10.6.4-focal
    command: '--default-authentication-plugin=mysql_native_password'
    volumes:
      - db_data:/var/lib/mysql
    restart: always
    environment:
      - MYSQL_ROOT_PASSWORD=somewordpress
      - MYSQL_DATABASE=wordpress
      - MYSQL_USER=wordpress
      - MYSQL_PASSWORD=wordpress
    expose:
      - 3306
      - 33060
  wordpress:
    image: wordpress:latest
    ports:
      - 80:80
    restart: always
    environment:
      - WORDPRESS_DB_HOST=db
      - WORDPRESS_DB_USER=wordpress
      - WORDPRESS_DB_PASSWORD=wordpress
      - WORDPRESS_DB_NAME=wordpress
volumes:
  db_data:
```

```yaml
services:
  portainer:
    image: portainer/portainer-ce:alpine
    container_name: portainer
    command: -H unix:///var/run/docker.sock
    ports:
      - "9000:9000"
    volumes:
      - "/var/run/docker.sock:/var/run/docker.sock"
      - "portainer_data:/data"
    restart: always

volumes:
  portainer_data:
```

```yaml
version: "3.9"
x-volumes:
  &default-volume
  driver: foobar-storage

services:
  web:
    image: myapp/web:latest
    volumes: ["vol1", "vol2", "vol3"]
volumes:
  vol1: *default-volume
  vol2:
    << : *default-volume
    name: volume02
  vol3:
    << : *default-volume
    driver: default
    name: volume-local
```